In [1]:
import cv2 as cv

In [2]:
## Importing Image
img = cv.imread("testImages/testIMG01.png")
cv.imshow("ImG", img)

cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
## Converting to Gray scale
gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)

cv.imshow("Gray", gray)

cv.waitKey(0)
cv.destroyAllWindows()

- Now we'll convert to binary Image (Black & White) to enhance the chance of text extraction using thresholding.  

- We also have to fine-tune the thresh values for better text recognition

In [4]:
ret, thresh = cv.threshold(gray, 165, 255, type=cv.THRESH_BINARY_INV)

cv.imshow("Thresh", thresh)

cv.waitKey(0)
cv.destroyAllWindows()

- To get the size of the words in the images we need a structure element method with kernel size depending up the area of the text.  
- We'll use the K-Size of (2,2) or (3, 3).  
- (3, 3) is the better one here.  
- 

In [5]:
rect_kernel = cv.getStructuringElement(cv.MORPH_RECT, (2, 2))

- We'll dilate the image to get the boundaries of the text(even if we don't see).  

In [27]:
dilation = cv.dilate(thresh, rect_kernel, iterations = 5)

cv.imshow("Dilation", dilation)

cv.waitKey(0)
cv.destroyAllWindows()

- Now we'll find the contours using `findContours` method.  
- We'll use the cv.RETR_EXTERNAL for finding the external contours of the

In [7]:
contours, hierarchy = cv.findContours(dilation, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

print(f'{len(contours)} contours found!')

149 contours found!


In [8]:
img2 = img.copy()

In [26]:
cv.imshow("2nd IMG", img2)
cv.waitKey(0)
cv.destroyAllWindows()

- Now we'll get the coordinates of the white pixel area and draw the bounding box around it.  
- We'll also save the text from the image  in the text file.  
- 

In [9]:
import pytesseract

In [10]:
from PIL import Image

We will try to increase the size and clear the image as a whole.  


In [ ]:
# Increasing the size of the img2 by 5-7 times

""" 
As we'll use Pillow library to resize the image and it recognizes RGB and not BGR.  
We'll convert BGR -> RGB
"""

# converting BGR -> RGB


img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)


# Converting the image to Pillow image object
img2_pil = Image.fromarray(img2)

# Getting the length and width of the image
length_x, width_y = img2_pil.size


# Increasing the size of the image by 5-7 times
size = int(5 * length_x), int(5 * width_y)

# Re-sizing the cropped image
img2_pil_resized = img2_pil.resize(size, )

#### Resizing the image using OpenCV only.

- We can also resize and scale the images using OpenCV.  
- We use `.resize()` function for this.  
- Only two inputs arguments are required:
    - The source image
    - the Desired Size of the resized image, `dsize`.
- The various input argument options for `.resize()` are:
    `resize(src, dsize, fx, fy, interpolation)`.  
    - `src`: the required input image.  
    - `dsize`: The desired size of the output image.  it can be a tuple of new height and new width.  
    - `fx`: Scale factor along the horizontal axis.  
    - `fy`: Scale factor along the vertical axis.  
    - `interpolation`: It give us the option of different methods of resizing of an image.  
---
- 

In [35]:
cv.imshow("Original", img)
cv.waitKey(0)
cv.destroyAllWindows()

In [58]:
from cv2 import INTER_CUBIC, INTER_LINEAR


scale_up = 2

img_scale_up_linear = cv.resize(img, None, fx=scale_up, fy=scale_up, interpolation= INTER_LINEAR)

In [59]:
from cv2 import INTER_CUBIC, INTER_LINEAR


scale_up = 2

img_scale_up_cubic = cv.resize(img, None, fx=scale_up, fy=scale_up, interpolation= INTER_CUBIC)

In [60]:
from cv2 import INTER_CUBIC, INTER_LANCZOS4, INTER_LINEAR


scale_up = 2

img_scale_up_lanczo = cv.resize(img, None, fx=scale_up, fy=scale_up, interpolation= INTER_LANCZOS4)

In [61]:
cv.imshow("Scaled Up (LINEAR)", img_scale_up_linear)
cv.imshow("Scaled Up (cubic)", img_scale_up_cubic)
cv.imshow("Scaled Up (Lanczo)", img_scale_up_lanczo)
cv.waitKey(0)
cv.destroyAllWindows()

In [62]:
# Removing the Noise
img_denoise= cv.fastNlMeansDenoisingColored(img_scale_up_lanczo, None, 10, 10,7, 15)



In [64]:
# Gray
gray_lanczo=cv.cvtColor(img_scale_up_lanczo, cv.COLOR_BGR2GRAY)
gray_denoise=cv.cvtColor(img_denoise, cv.COLOR_BGR2GRAY)

In [65]:
# cv.imshow("Scaled Up (LINEAR)", img_scale_up_linear)
# cv.imshow("Scaled Up (cubic)", img_scale_up_cubic)
cv.imshow("Scaled Up (Lanczo)", img_scale_up_lanczo)
cv.imshow("Noise Removed", img_denoise)

# Gray
# gray_lanczo=cv.cvtColor(img_scale_up_lanczo, cv.COLOR_BGR2GRAY)
# gray_denoise=cv.cvtColor(img_denoise, cv.COLOR_BGR2GRAY)

cv.imshow("gray_lanczoy", gray_lanczo)
cv.imshow("gray_denoise", gray_denoise)


cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
gray_lanczo=cv.cvtColor(img_scale_up_lanczo, cv.COLOR_BGR2GRAY)
gray_denoise=cv.cvtColor(img_denoise, cv.COLOR_BGR2GRAY)

cv.imshow("gray_lanczoy", gray_lanczo)
cv.imshow("gray_denoise", gray_denoise)

cv.waitKey(0)
cv.destroyAllWindows()

In [50]:
print(type(contours))

<class 'tuple'>


In [33]:

import numpy as np


crop_number = 0
for cnt in contours:
    x,y, w, h = cv.boundingRect(cnt)
    
    # Draw the bounding box on the text area
    rect = cv.rectangle(img2, (x-2, y-2), (x+w, y+h), (0, 255, 2), 1)
    
    
    # Crop the bounding box area
    cropped = img2[y-1:y + h, x-1:x+w]

    """ 
    OpenCv read the color in BGR format, thus we've to convert it to RGB for it to be properly scaling using Pillow.  
    We'll convert it to a pillow image object.  
    Then we'll resize it using the resize method in the PIL.  
    - Convert back to an np array & then back to BGR 
    
    - Now our image is a binary image (Thresholded image) thus we don't have to follow the conversion of BGR to RGB and visa versa. (NO NOT Follow this as we're applying these to img2 and not on img)
    """
    
    # converting BGR -> RGB
    cropped_RGB = cv.cvtColor(cropped, cv.COLOR_BGR2RGB)
    
    # Converting image to PIL image object
    cropped_pil = Image.fromarray(cropped_RGB)
    
    # Getting the length and width of the image
    length_x, width_y = cropped_pil.size
    
    
    # Increasing the size(length & width) by 5
    size = int(7 * length_x), int(7 * width_y)
    

    """
    length_x, width_y = cropped.shape[:2]
    
    # length_x = cropped.shape[0]
    # width_y = cropped.shape[1]
    
    size = (5 * int(length_x)), (5 * int(width_y))
    """
    
    # Resizing the cropped image
    cropped_pil_resized = cropped_pil.resize(size, Image.LANCZOS)
    
    
    # Converting the cropped_pil_resized to cropped_np
    cropped_resized = np.array(cropped_pil_resized)
    
    cropped_resized = cv.cvtColor(cropped_resized, cv.COLOR_RGB2BGR)
    
    
    
    
    # Saving the Cropped images as a .jpeg file
    cv.imwrite("Crop_resized"+str(crop_number)+".jpeg", cropped_resized)
    crop_number+=1
    
    # Open the text file
    file = open("Text_Output2.txt", "a")
    
    
    
    
    # cv.imwrite("rectangleBox.jpg", rect)
    
    text = pytesseract.image_to_string(cropped_resized)

    # Adding the text to the file
    file.write(text)
    file.write("\n")
    
    # Closing the file
    file.close()
    
    # print(text)
    
cv.imshow("BoundBoxed", rect)
cv.imshow("Dilation", dilation)
cv.waitKey(0)
cv.destroyAllWindows()

In [34]:
cv.imshow("BoundBoxed", rect)
cv.waitKey(0)
cv.destroyAllWindows()

# Image Preprocessing to improve the quality of the image
